In [1]:
# ***IMPORTANT!:
# first install python-binance package (using "pip") into this anaconda environment:

# conda activate cryptoalgowheel
# /anaconda3/envs/cryptoalgowheel/bin/pip install python-binance

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.tz import tzutc
import dateparser   #important datetime-related package used in "python-binance" SDK: https://dateparser.readthedocs.io/en/latest/ 

In [3]:
from binance.client import Client
client = Client()

In [5]:
# client.get_order_book(symbol="BNBBTC")["bids"]

In [6]:
client.get_all_tickers()

,
 {'symbol': 'WINBNB', 'price': '0.00000479'},
 {'symbol': 'WINBTC', 'price': '0.00000001'},
 {'symbol': 'WINUSDT', 'price': '0.00010970'},
 {'symbol': 'WINUSDC', 'price': '0.00010990'},
 {'symbol': 'COSBNB', 'price': '0.00031900'},
 {'symbol': 'COSBTC', 'price': '0.00000072'},
 {'symbol': 'COSUSDT', 'price': '0.00730000'},
 {'symbol': 'TUSDBTUSD', 'price': '0.99750000'},
 {'symbol': 'NPXSUSDT', 'price': '0.00015120'},
 {'symbol': 'NPXSUSDC', 'price': '0.00017020'},
 {'symbol': 'COCOSBNB', 'price': '0.00001340'},
 {'symbol': 'COCOSBTC', 'price': '0.00000003'},
 {'symbol': 'COCOSUSDT', 'price': '0.00030700'},
 {'symbol': 'MTLUSDT', 'price': '0.30070000'},
 {'symbol': 'TOMOBNB', 'price': '0.02900000'},
 {'symbol': 'TOMOBTC', 'price': '0.00006500'},
 {'symbol': 'TOMOUSDT', 'price': '0.66660000'},
 {'symbol': 'TOMOUSDC', 'price': '0.36700000'},
 {'symbol': 'PERLBNB', 'price': '0.00137000'},
 {'symbol': 'PERLBTC', 'price': '0.00000307'},
 {'symbol': 'PERLUSDC', 'price': '0.02447000'},
 {'s

In [7]:
# see "client.py" source code - line 714

np.array(client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")).shape
test = np.array(client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago"))   #by default starting from UTC ("time value" in terms of current time zone) thus 28 hours covered in this case
test.shape
# test


(1680, 12)

In [8]:
test = np.array(client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago EDT"))
test = test[:,:9]
test = pd.DataFrame(test, columns=["OpenTime", "Open", "High", "Low", "Close", "Volume", "CloseTime", "QuoteAssetVol", "NumOfTrades"])
# print(test.dtypes, "\n")
test.OpenTime = test.OpenTime.astype("int")
# print(test.dtypes)
test

,OpenTime,Open,High,Low,Close,Volume,CloseTime,QuoteAssetVol,NumOfTrades
0,1599355260000,0.00188920,0.00188920,0.00188560,0.00188920,870.97000000,1599355319999,1.64385769,40
1,1599355320000,0.00188890,0.00188890,0.00188680,0.00188750,20.07000000,1599355379999,0.03788402,11
2,1599355380000,0.00188720,0.00188820,0.00188590,0.00188590,696.46000000,1599355439999,1.31450078,30
3,1599355440000,0.00188580,0.00188930,0.00188580,0.00188720,2418.10000000,1599355499999,4.56663900,82
4,1599355500000,0.00188620,0.00189000,0.00188580,0.00188890,1567.93000000,1599355559999,2.96143314,46
...,...,...,...,...,...,...,...,...,...
1435,1599441360000,0.00223710,0.00223710,0.00223370,0.00223370,913.13000000,1599441419999,2.04065846,104
1436,1599441420000,0.00223410,0.00223410,0.00222620,0.00222690,2835.72000000,1599441479999,6.32226063,395
1437,1599441480000,0.00222720,0.00223490,0.00222670,0.00223300,4168.69000000,1599441539999,9.29897294,157
1438,1599441540000,0.00223290,0.00223410,0.00223050,0.00223300,1906.32000000,1599441599999,4.25793471,43


In [9]:
datetime.utcfromtimestamp(1599364800000/1000).strftime("%Y/%m/%d %H:%M UTC")

'2020/09/06 04:00 UTC'

In [10]:
dateparser.parse("2 minutes ago")

datetime.datetime(2020, 9, 6, 21, 18, 19, 286753)

### Candle Data

*see "client.py" source code - starting line 673* <br>
**generally "OHLCV" kind of data result format (12 fields):** <br>
Open time; <br>
Open; <br>
High; <br>
Low; <br>
Close; <br>
Volume; <br>
Close time; <br>
Quote asset volume; <br>
Number of trades; <br>
Taker buy base asset volume; <br>
Taker buy quote asset volume; <br>
a_field_that_can_be_ignored


In [11]:
def get_candle(symbol, interval_str, start_time, end_time=None):
    client = Client()
    result = client.get_historical_klines(symbol=symbol, interval=interval_str, start_str=start_time, end_str=end_time)     #return two dimensional lists with each row having 12 fields ("OHLCV" kind of data)
    result = np.array(result)
    result = result[:,:9]
    result = pd.DataFrame(result, columns=["OpenTime", "Open", "High", "Low", "Close", "Volume", "CloseTime", "QuoteAssetVol", "NumOfTrades"])
    result.OpenTime = result.OpenTime.astype("int").apply(lambda d: datetime.utcfromtimestamp(d/1000).strftime("%Y/%m/%d %H:%M:%S UTC"))       #convert "OpenTime" to time string format (notice that original UNIX timestamp is in milliseconds!)
    result.CloseTime = result.CloseTime.astype("int").apply(lambda d: datetime.utcfromtimestamp(d/1000).strftime("%Y/%m/%d %H:%M:%S UTC")) #convert "CloseTime" to time string format
    result = result[["OpenTime", "CloseTime", "Open", "High", "Low", "Close", "Volume", "QuoteAssetVol", "NumOfTrades"]]
    result.to_csv("/Users/baixiao/Desktop/histo_kline_data.csv", index=False)


In [13]:
get_candle("BNBBTC", Client.KLINE_INTERVAL_1HOUR, "1 day ago EDT")
#get_candle("NEOBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago EDT")

### Transactions

In [14]:
#client=Client()
#try:
#    client.get_aggregate_trades(symbol = "BNBBTC", startTime = 1599350400000, endTime = 1599411600000)
#except:
#    print('Wrong!')

In [15]:
pd.DataFrame(client.get_aggregate_trades(symbol = "BNBBTC", startTime = 1599411600000, endTime = 1599413400000))

,a,p,q,f,l,T,m,M
0,71422683,0.00226000,8.84000000,86866793,86866793,1599411602346,True,True
1,71422684,0.00225990,8.14000000,86866794,86866794,1599411602359,True,True
2,71422685,0.00226050,0.17000000,86866795,86866795,1599411603619,False,True
3,71422686,0.00226010,10.00000000,86866796,86866796,1599411604376,False,True
4,71422687,0.00226020,14.98000000,86866797,86866797,1599411604376,False,True
...,...,...,...,...,...,...,...,...
2959,71425642,0.00224080,1.42000000,86870390,86870390,1599413396770,True,True
2960,71425643,0.00224070,0.07000000,86870391,86870391,1599413396804,True,True
2961,71425644,0.00224050,1.02000000,86870392,86870393,1599413396839,True,True
2962,71425645,0.00224030,1.99000000,86870394,86870394,1599413396839,True,True


In [16]:
dateparser.parse("January 12, 2012 10:00PM EST").timestamp()*1000

1326423600000.0

#### aggregated trades list

##### API return: <br>
Aggregate tradeId; <br>
Price; <br>
Quantity; <br>
First tradeId; <br>
Last tradeId; <br>
Timestamp; <br>
Was_the_buyer_the_maker?; <br>
Was_the_trade_the_best_price_match?

In [17]:
#API method see "client.py" source code - line 539

def get_agg_trades(symbol, start_time=None, end_time=None, fromId=None):
    #convert user-input datetime string format into UNIX timestamp:
    if isinstance(start_time, str):
        start_time = int(dateparser.parse(start_time).timestamp()*1000)
    if isinstance(end_time, str):
        end_time = int(dateparser.parse(end_time).timestamp()*1000)

    client = Client()
    try:
        result = client.get_aggregate_trades(symbol=symbol, startTime=start_time, endTime=end_time, fromId=fromId)
        result = pd.DataFrame(result)
        result = result.rename(columns={"a":"agg_tradeId", "p":"price", "q":"quantity", "f":"first_tradeId", "l":"last_tradeId", "T":"timestamp", "m":"isBuyerMaker", "M":"isBestMatch"})
        result.timestamp = result.timestamp.apply(lambda d: datetime.utcfromtimestamp(d/1000).strftime("%Y/%m/%d %H:%M:%S UTC"))       #convert "timestamp" to time string format
        result = result[["timestamp", "agg_tradeId", "price", "quantity", "first_tradeId", "last_tradeId", "isBuyerMaker", "isBestMatch"]]
        result.to_csv("/Users/baixiao/Desktop/agg_trades.csv", index=False)
    except:
        print("Error: time between startTime and endTime passed in cannot exceed 1 hour here!!")

In [18]:
#get_agg_trades("BNBBTC", 1599411600000, 1599413400000)
#get_agg_trades("BNBBTC", 1599350400000, 1599411600000)  #(*)Error appearing

get_agg_trades("BNBBTC", "September 6, 2020 13:00 EDT", "September 6, 2020 13:30 EDT")

#### most recent grades (last 500)

In [19]:
#API method see "client.py" source code - line 477

def get_rct_trades(symbol):
    client = Client()
    result = client.get_recent_trades(symbol=symbol)
    result = pd.DataFrame(result)
    result = result.rename(columns={"qty":"quantity"})
    result.time = result.time.apply(lambda d: datetime.utcfromtimestamp(d/1000).strftime("%Y/%m/%d %H:%M:%S UTC"))       #convert "time" to time string format
    result = result[["time", "id", "price", "quantity", "isBuyerMaker", "isBestMatch"]]
    result.to_csv("/Users/baixiao/Desktop/latest_500_trades.csv", index=False)


In [20]:
get_rct_trades("NEOBTC")

### Market Depth (get latest orderbook entries)

In [21]:
client = Client()
c = client.get_order_book(symbol="BNBBTC")
# pd.DataFrame(c)
np.array(c["bids"])[:,0]
temp = pd.DataFrame()
temp["bids_price"] = np.array(c["bids"])[:,0]
temp["bids_price"] = temp["bids_price"].astype("float")
temp.dtypes

bids_price    float64
dtype: object

In [22]:
#api method see "client.py" source code - line 440

def get_orderbook(symbol, limit=500):
    client=Client()
    api_result = client.get_order_book(symbol=symbol, limit=limit)
    result=pd.DataFrame()
    result["bid_price"] = np.array(api_result["bids"])[:,0]
    result["bid_quantity"] = np.array(api_result["bids"])[:,1]
    result["ask_price"] = np.array(api_result["asks"])[:,0]
    result["ask_quantity"] = np.array(api_result["asks"])[:,1]
    result.index += 1
    result.to_csv("/Users/baixiao/Desktop/latest_orderbook.csv", index=True)

In [23]:
get_orderbook("NEOBTC")